<a href="https://colab.research.google.com/github/jisu1013/DL_starting_with_PyTorch/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
data = pd.read_table('ratings.txt')
data[:10]

In [ ]:
print('전체 샘플의 수: {}'.format(len(data)))

In [ ]:
sample_data = data[:100] #임의로 100개만 저장

정규 표현식을 통해서 데이터를 정제.

In [ ]:
sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
sample_data[:10]

토큰화 수행. 우선 불용어를 제거하기 위해서 불용어를 우선 정의한다.

In [ ]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

형태소 분석기는 mecab을 사용한다.

In [ ]:
tokenizer = Mecab()

In [ ]:
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) #토큰화
    temp = [word for word in temp if not word in stopwords] #불용어 제거
    tokenized.append(temp)

In [ ]:
print(tokenized[:10])

단어 집합을 만들어보자. NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원한다.

In [ ]:
vocab = FreqDist(np.hstack(tokenized))
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어를 key로, 단어에 대한 빈도수가 value로 저장되어 있다. vocab에 단어를 입력하면 빈도수를 리턴한다.

In [ ]:
vocab['재밌']

'재밌'이란 단어가 총 10번 등장함.
most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴. 

이를 사용하여 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있다. 

등장 빈도수 상위 500개의 단어만 단어 집합으로 저장해보자 !!

In [ ]:
vocab_size = 500
# 상위 vocab_size개의 단어만 보존
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기 : {}'.format(len(vocab)))

### 각 단어에 고유한 정수 부여
enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아서 인덱스를 순차적으로 함께 리턴한다는 특징이 있다. 인덱스 0과 1은 다른 용도로 남겨 두고 나머지 단어들은 2부터 501까지 순차적으로 인덱스를 부여.

In [ ]:
word_to_index = {word[0] : index + 2 for index, word in enumerate(vocab)}
word_to_index['pad'] = 1
word_to_index['unk'] = 0

이제 기존의 훈련 데이터에서 각 단어를 고유한 정수로 부여하는 작업을 진행.

In [ ]:
encoded = []
for line in tokenized:
    temp = []
    for w in line:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['unk'])
    encoded.append(temp)

In [ ]:
print(encoded[:10])

###길이가 다른 문장들을 모두 동일한 길이로 바꿔주는 패딩(padding)

앞서 단어 집합에 패딩을 위한 토큰인 'pad'를 추가.
패딩 작업은 정해준 길이로 모든 샘플들의 길이를 맞춰주되, 길이가 정해준 길이보다 짧은 샘플들에는 'pad' 토큰을 추가하여 길이를 맞춰주는 작업.

In [ ]:
max_len = max(len(i) for i in encoded)
print('리뷰의 최대길이 : %d' % max_len)
print('최소 길이:  %d' % min(len(l) for l in encoded))
print('리뷰의 평균 길이 : %f' % (sum(map(len, encoded))/len(encoded)))
plt.hist([len(s) for s in encoded], bins=50)
plt.xlabel('length of sample')
plt.ylabel('number of sample')
plt.show()

모든 리뷰의 길이를 63으로 통일